In [1]:
%pip install torcheeg
%pip install moabb==0.5


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from torcheeg.datasets import BCICIV2aDataset
from torcheeg import transforms
from typing import Dict, Union
import numpy as np
from torcheeg.transforms import EEGTransform

# Custom transform to traspose matrix

class TrasposeEEG(EEGTransform):
    def __init__(self, apply_to_baseline: bool = False):
        super(TrasposeEEG, self).__init__(apply_to_baseline=apply_to_baseline)

    def __call__(self,
                 *args,
                 eeg: np.ndarray,
                 baseline: Union[np.ndarray, None] = None,
                 **kwargs) -> Dict[str, np.ndarray]:
        return super().__call__(*args, eeg=eeg, baseline=baseline, **kwargs)
    
    def apply(self, eeg: np.ndarray, **kwargs) -> np.ndarray:
        return np.moveaxis(eeg, -1, -2)


In [3]:
# dataset = moabb_dataset.MOABBDataset(
#     dataset=moabb,
#     paradigm=paradigm,
#     io_path='./io/moabb',
#     # offline_transform=transforms.Compose([transforms.BandDifferentialEntropy()]),
#     online_transform=transforms.ToTensor(),
#     label_transform=transforms.Compose([
#         transforms.Select('label'),
#         transforms.Mapping({'left_hand': 0, 'right_hand': 1,}),
#     ])
# )

# Load BCI-Competition IV-2a dataset

SEQ_LENGTH = 500
 
dataset = BCICIV2aDataset(root_path='../datasets/bci_c',
                          io_path='.torcheeg/datasets_1706557112340_X93HI',
                          chunk_size=SEQ_LENGTH,
                          online_transform=transforms.Compose([
                              transforms.To2d(),
                              TrasposeEEG(),
                              transforms.ToTensor()
                          ]),
                          label_transform=transforms.Compose([
                              transforms.Select('label'),
                              transforms.Lambda(lambda x: x - 1)
                          ]))
print(dataset[0])

[2024-01-29 22:39:16] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from .torcheeg/datasets_1706557112340_X93HI.


(tensor([[[13.1348, 13.8672, 17.0898,  ...,  9.0820,  7.2266,  3.3691],
         [14.5996, 18.7988, 17.9199,  ..., 14.8926, 12.6465, 10.8398],
         [16.1621, 18.1641, 19.2871,  ..., 15.2832, 14.0625, 11.4746],
         ...,
         [-3.4180, -8.9355, -7.8613,  ..., -2.7832,  0.5371, -4.3945],
         [ 1.0254, -3.0273, -1.8555,  ...,  1.6602,  3.8086, -0.4395],
         [ 1.5625, -0.2930,  0.4395,  ...,  4.2969,  6.2500,  2.3438]]]), 0)


In [4]:
dataset[2][0].shape

torch.Size([1, 500, 22])

In [5]:
import torch


import torch.nn as nn
from ncps.wirings import AutoNCP
from ncps.torch import CfC

class OnlyLiquidEEG(nn.Module):
    def __init__(self, liquid_units=50, num_classes=2, channels=4):
        super().__init__()
        self.liquid_block = LiquidBlock(units=liquid_units, out_features=num_classes, in_features=channels)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x, state=None):
        x, _ = self.liquid_block.forward(torch.squeeze(x, dim=1), state)
        x = self.softmax(x)

        return x

class LiquidBlock(nn.Module):
    def __init__(self, units=20, out_features=10, in_features=5):
        super().__init__()
        wiring = AutoNCP(units, out_features)
        self.units = units
        self.liquid = CfC(in_features, wiring, return_sequences=False, batch_first=True)

    def forward(self, x, state=None):
        x, hx = self.liquid.forward(input=x, hx=state)
        return x, hx
    
class ConvolutionalBlock(nn.Module):
    def __init__(self, dropout):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=(1, 2)),

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=3, stride=(1, 2)),

            nn.Conv2d(in_channels=64, out_channels=1, kernel_size=5, stride=1, padding='same'),
            nn.BatchNorm2d(1),
            nn.ReLU(),
            # nn.Dropout2d(p=dropout),
            nn.MaxPool2d(kernel_size=(1, 3), stride=(1, 2))
        )
    
    def forward(self, x):
        return self.cnn(x)
    
class ConvLiquidEEG(nn.Module):
    # TODO: Prepare model for ablation tests
    def __init__(self, liquid_units=20, num_classes=10, dropout=0):
        super().__init__()
        self.conv_block = ConvolutionalBlock(dropout=dropout)
        # TODO Parametrize in features
        self.liquid_block = LiquidBlock(units=liquid_units, out_features=num_classes, in_features=4)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x, state=None):
        # print(f'>>>>>>> INPUT: {x.shape}')
        # x = self.preprocessing(x)
        # print(f'>>>>>>>>> PRE OUTPUT: {x.shape}')
        x = self.conv_block(x)
        # print(f'>>>>>>>>>> CONV OUTPUT SHAPE: {x.shape}')
        x, _ = self.liquid_block.forward(torch.squeeze(x, dim=1))
        # print(f'>>>>>>>>> LIQUID OUT: {x.shape}')
        return self.softmax(x)
    
class ConvLSTMEEG(nn.Module):
    def __init__(self, num_classes=10, dropout=0):
        self.conv_block = ConvolutionalBlock(dropout=dropout)
        self.lstm = nn.LSTM(4, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv_block(x)
        x, _ = self.liquid_block.forward(torch.squeeze(x, dim=1))

In [6]:
import os
import time
import logging

os.makedirs('./examples_vanilla_torch/log', exist_ok=True)
logger = logging.getLogger('Training models with vanilla PyTorch')
logger.setLevel(logging.DEBUG)
console_handler = logging.StreamHandler()
timeticks = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
file_handler = logging.FileHandler(
    os.path.join('./examples_vanilla_torch/log', f'{timeticks}.log'))
logger.addHandler(console_handler)
logger.addHandler(file_handler)

In [7]:
import random
import numpy as np

import torch

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(42)

In [8]:
from torcheeg.model_selection import KFoldPerSubject, train_test_split

k_fold = KFoldPerSubject(n_splits=10,
                         split_path='./examples_vanilla_torch/split',
                         shuffle=True)

In [9]:
# import datetime

# device = "cuda" if torch.cuda.is_available() else "cpu"

# # training process
# def train(dataloader, model, loss_fn, optimizer):
#     # Set the model to training mode - important for batch normalization and dropout layers
#     # Unnecessary in this situation but added for best practices
#     model.train()
#     total_start = datetime.datetime.now()
#     size = len(dataloader.dataset)

#     last_batch = 0
#     start_batch_loading = datetime.datetime.now()
#     for batch_idx, (X, y) in enumerate(dataloader):
#         start_batch_training = datetime.datetime.now()
#         batch_loading_time = (start_batch_training - start_batch_loading).total_seconds()
  
#         X, y = X.to(device), y.to(device)

#         # Compute prediction and loss
#         pred = model(X)
#         loss = loss_fn(pred, y)

#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         batch_training_time = (datetime.datetime.now() - start_batch_training).total_seconds()
#         start_batch_loading = datetime.datetime.now()
#         if batch_idx % 50 == 0:
#             loss, current = loss.item(), batch_idx * len(X)
#             logger.info(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}] training time ratio: {batch_training_time / (batch_training_time + batch_loading_time)}")
#     last_batch += 1
#     print(f'Training time: {(datetime.datetime.now() - total_start).total_seconds()}')
    
#     return loss


# def valid(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     model.eval()
#     loss, correct = 0, 0
#     with torch.no_grad():
#         for batch, (X, y) in enumerate(dataloader):
#             X, y = X.to(device), y.to(device)

#             pred = model(X)
#             loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()

#     loss /= num_batches
#     correct /= size
#     logger.info(f"Valid Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f}\n")

#     return correct, loss

In [10]:
# import torch.nn as nn

# from torcheeg.model_selection import train_test_split
# from torch.utils.data.dataloader import DataLoader

# loss_fn = nn.CrossEntropyLoss()
# batch_size = 64
# NUM_CLASSES = 4

# test_accs = []
# test_losses = []

# for i, (train_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
#     # initialize model
#     model = ConvLiquidEEG(liquid_units=250, eeg_channels=22, num_classes=NUM_CLASSES, dropout=0).to(device)
#     # initialize optimizer
#     optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # official: weight_decay=5e-1
#     # split train and val
#     train_dataset, val_dataset = train_test_split(
#         train_dataset,
#         test_size=0.2,
#         split_path=f'./examples_vanilla_torch/split{i}',
#         shuffle=True)
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
#     val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

#     epochs = 5
#     best_val_acc = 0.0
#     for t in range(epochs):
#         train_loss = train(train_loader, model, loss_fn, optimizer)
#         val_acc, val_loss = valid(val_loader, model, loss_fn)
#         # save the best model based on val_acc
#         if val_acc > best_val_acc:
#             best_val_acc = val_acc
#             torch.save(model.state_dict(),
#                        f'./examples_vanilla_torch/model{i}.pt')

#     test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#     # load the best model to test on test set
#     model.load_state_dict(torch.load(f'./examples_vanilla_torch/model{i}.pt'))
#     test_acc, test_loss = valid(test_loader, model, loss_fn)

#     # log the test result
#     logger.info(
#         f"Test Error {i}: \n Accuracy: {(100*test_acc):>0.1f}%, Avg loss: {test_loss:>8f}"
#     )

#     test_accs.append(test_acc)
#     test_losses.append(test_loss)

# # log the average test result on cross-validation datasets
# logger.info(
#     f"Test Error: \n Accuracy: {100*np.mean(test_accs):>0.1f}%, Avg loss: {np.mean(test_losses):>8f}"
# )

In [11]:
from torch.utils.data import DataLoader
from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

NUM_CLASSES = 4

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=7)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

    # model = ConvLiquidEEG(liquid_units=250, num_classes=NUM_CLASSES)
    model = ConvLiquidEEG(num_classes=NUM_CLASSES)


    trainer = ClassifierTrainer(model=model,
                                num_classes=NUM_CLASSES,
                                lr=1e-3,
                                weight_decay=1e-5,
                                accelerator="gpu")
    trainer.fit(train_loader,
                val_loader,
                max_epochs=3,
                default_root_dir=f'./examples_pipeline/model/{i}',
                callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
                enable_progress_bar=True,
                enable_model_summary=True,
                limit_val_batches=0.0)
    score = trainer.test(val_loader,
                         enable_progress_bar=True,
                         enable_model_summary=True)[0]
    print(f'Fold {i} test accuracy: {score["test_accuracy"]:.4f}')


[2024-01-29 22:39:16] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from ./examples_vanilla_torch/split.
[2024-01-29 22:39:16] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2024-01-29 22:39:16.963485: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Netwo

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
